In [6]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [7]:
# Define the directories and categories
original_dataset_dir = '/Users/code/alzheimerDetect/OriginalDataset'
categories = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

In [8]:
# Parameters
img_size = 128  

In [9]:
# Load images and labels
data = []
labels = []

for category in categories:
    category_path = os.path.join(original_dataset_dir, category)
    class_num = categories.index(category)
    for img in os.listdir(category_path):
        try:
            img_path = os.path.join(category_path, img)
            img_array = Image.open(img_path).convert('L') 
            resized_array = img_array.resize((img_size, img_size))
            data.append(np.array(resized_array))
            labels.append(class_num)
        except Exception as e:
            print(f"Error loading image {img}: {e}")

In [10]:
# Convert lists to numpy arrays and normalize pixel values
data = np.array(data).reshape(-1, img_size, img_size, 1) / 255.0
labels = np.array(labels)

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [13]:
# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 21s 0us/step


In [14]:
# Add a flatten layer to convert the 3D output to 1D
x = Flatten()(base_model.output)
feature_extractor = Model(inputs=base_model.input, outputs=x)

In [15]:
# Extract features for training and testing data
X_train_features = feature_extractor.predict(np.repeat(X_train, 3, axis=-1))
X_test_features = feature_extractor.predict(np.repeat(X_test, 3, axis=-1))

2024-07-31 23:07:25.546050: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


160/160 ━━━━━━━━━━━━━━━━━━━━ 19s 110ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 111ms/step


In [16]:

# Initialize and train the SVM
svm = SVC(kernel='linear')
svm.fit(X_train_features, y_train)


SVC(kernel='linear')

In [17]:
# Predict on the test set
y_pred = svm.predict(X_test_features)

In [18]:
# Print the classification report and accuracy
print(classification_report(y_test, y_pred, target_names=categories))
print("Accuracy:", accuracy_score(y_test, y_pred))

                  precision    recall  f1-score   support

    MildDemented       0.89      0.97      0.93       201
ModerateDemented       1.00      1.00      1.00         6
     NonDemented       0.96      0.93      0.95       643
VeryMildDemented       0.92      0.92      0.92       430

        accuracy                           0.94      1280
       macro avg       0.94      0.96      0.95      1280
    weighted avg       0.94      0.94      0.94      1280

Accuracy: 0.9359375


In [19]:
import joblib

# Save the trained SVM model
joblib.dump(svm, 'svm_alzheimer_model.pkl')

['svm_alzheimer_model.pkl']

In [20]:
# Load the saved SVM model
svm_loaded = joblib.load('svm_alzheimer_model.pkl')

In [22]:
def preprocess_image(image_path, img_size=128):
    img_array = Image.open(image_path).convert('L')  
    resized_array = img_array.resize((img_size, img_size))
    normalized_array = np.array(resized_array).reshape(1, img_size, img_size, 1) / 255.0
    return normalized_array

def extract_features(image_array, feature_extractor):
    return feature_extractor.predict(np.repeat(image_array, 3, axis=-1))

def predict_image(image_path, model, feature_extractor):
    # Preprocess the image
    preprocessed_image = preprocess_image(image_path)
    
    # Extract features
    features = extract_features(preprocessed_image, feature_extractor)
    
    # Predict using the loaded model
    prediction = model.predict(features)
    
    return prediction

# Load the VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
x = Flatten()(base_model.output)
feature_extractor = Model(inputs=base_model.input, outputs=x)

# Predict the class of a new image
image_path = '/Users/arnabsahoo/Downloads/FinalPriject/Alzheimer_s Dataset/test/VeryMildDemented/26 (53).jpg'  
prediction = predict_image(image_path, svm_loaded, feature_extractor)

# Convert the numerical prediction to the corresponding category
predicted_category = categories[prediction[0]]
print("Predicted Category:", predicted_category)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
Predicted Category: VeryMildDemented
